In [1]:
pip install wooldridge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 30.7 MB/s eta 0:00:00


In [2]:
pip install linearmodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00


In [3]:
pip install pandas statsmodels

In [4]:
import wooldridge as wd
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
from linearmodels.panel import PanelOLS

# **C3**

In [5]:
df = wd.data('jtrain')
wd.data('jtrain', description=True)

name of dataset: jtrain
no of variables: 30
no of observations: 471

+----------+---------------------------------+
| variable | label                           |
+----------+---------------------------------+
| year     | 1987, 1988, or 1989             |
| fcode    | firm code number                |
| employ   | # employees at plant            |
| sales    | annual sales, $                 |
| avgsal   | average employee salary         |
| scrap    | scrap rate (per 100 items)      |
| rework   | rework rate (per 100 items)     |
| tothrs   | total hours training            |
| union    | =1 if unionized                 |
| grant    | = 1 if received grant           |
| d89      | = 1 if year = 1989              |
| d88      | = 1 if year = 1988              |
| totrain  | total employees trained         |
| hrsemp   | tothrs/totrain                  |
| lscrap   | log(scrap)                      |
| lemploy  | log(employ)                     |
| lsales   | log(sales)               

In [6]:
periodos = pd.Categorical(df['fcode'])
#preparar painel para regressão
df = df.set_index(['fcode', 'year'])
#isso faz com que cada linha seja identificada de forma única por uma combinação de fcode e year
df['fcode'] = periodos
df

employ       sales   avgsal  scrap  rework  tothrs  union  \
fcode    year                                                              
410032.0 1987   100.0  47000000.0  35000.0    NaN     NaN    12.0      0   
         1988   131.0  43000000.0  37000.0    NaN     NaN     8.0      0   
         1989   123.0  49000000.0  39000.0    NaN     NaN     8.0      0   
410440.0 1987    12.0   1560000.0  10500.0    NaN     NaN    12.0      0   
         1988    13.0   1970000.0  11000.0    NaN     NaN    12.0      0   
...               ...         ...      ...    ...     ...     ...    ...   
419483.0 1988   108.0  11500000.0  14810.0   25.0     NaN     0.0      1   
         1989   129.0  12000000.0  14227.0   30.0     NaN    20.0      1   
419486.0 1987    80.0   7000000.0  16000.0    NaN     NaN     0.0      0   
         1988    90.0   8500000.0  17000.0    NaN     NaN     0.0      0   
         1989   100.0   9900000.0  18000.0    NaN     NaN    40.0      0   

               grant  d89  d88  ...   clscrap  cgrant  clemploy   clsales  \
fcode    year                   ...                                         
410032.0 1987      0    0    0  ...       NaN       0       NaN       NaN   
         1988      0    0    1  ...       NaN       0  0.270027 -0.088949   
         1989      0    1    0  ...       NaN       0 -0.063013  0.130621   
410440.0 1987      0    0    0  ...       NaN       0       NaN       NaN   
         1988      0    0    1  ...       NaN       0  0.080043  0.233347   
...              ...  ...  ...  ...       ...     ...       ...       ...   
419483.0 1988      0    0    1  ...  0.223144       0 -0.208218  0.044453   
         1989      0    1    0  ...  0.182322       0  0.177681  0.042559   
419486.0 1987      0    0    0  ...       NaN       0       NaN       NaN   
         1988      0    0    1  ...       NaN       0  0.117783  0.194157   
         1989      1    1    0  ...       NaN       1  0.105361  0.152469   

                 lavgsal  clavgsal  cgrant_1    chrsemp  clhrsemp     fcode  
fcode    year                                                                
410032.0 1987  10.463103       NaN       NaN        NaN       NaN  410032.0  
         1988  10.518673  0.055570       0.0  -8.946565 -1.165385  410032.0  
         1989  10.571317  0.052644       0.0   0.198597  0.047832  410032.0  
410440.0 1987   9.259130       NaN       NaN        NaN       NaN  410440.0  
         1988   9.305651  0.046520       0.0   0.000000  0.000000  410440.0  
...                  ...       ...       ...        ...       ...       ...  
419483.0 1988   9.603058  0.059321       0.0   0.000000  0.000000  419483.0  
         1989   9.562897 -0.040161       0.0   3.100775  1.411176  419483.0  
419486.0 1987   9.680344       NaN       NaN        NaN       NaN  419486.0  
         1988   9.740969  0.060625       0.0   0.000000  0.000000  419486.0  
         1989   9.798127  0.057158       0.0  36.000000  3.610918  419486.0  

[471 rows x 29 columns]

In [7]:
#estimando o modelo FE:
#modelo
y = df['hrsemp']
X = df[['d88', 'd89', 'grant', 'grant_1', 'lemploy']]
#adicionando intercepto
X = sm.add_constant(X)
#modelo de efeitos fixos
modelo = PanelOLS(y, X, entity_effects=True).fit()
print(modelo.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 hrsemp   R-squared:                        0.4909
Estimator:                   PanelOLS   R-squared (Between):              0.0481
No. Observations:                 390   R-squared (Within):               0.4909
Date:                Sun, May 26 2024   R-squared (Overall):              0.2201
Time:                        01:33:27   Log-likelihood                   -1503.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      48.206
Entities:                         135   P-value                           0.0000
Avg Obs:                       2.8889   Distribution:                   F(5,250)
Min Obs:                       1.0000                                           
Max Obs:                       3.0000   F-statistic (robust):             48.206
                            

/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:1219: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


# **C12**

In [32]:
df = wd.data('elem94_95')
wd.data('elem94_95', description=True)

name of dataset: elem94_95
no of variables: 14
no of observations: 1848

+----------+----------------------------------------+
| variable | label                                  |
+----------+----------------------------------------+
| distid   | district identifier                    |
| schid    | school identifier                      |
| lunch    | percent eligible, free lunch           |
| enrol    | enrollment                             |
| staff    | staff per 1000 students                |
| exppp    | expenditures per pupil                 |
| avgsal   | average teacher salary, $              |
| avgben   | average teacher non-salary benefits, $ |
| math4    | percent passing 4th grade math test    |
| story4   | percent passing 4th grade reading test |
| bs       | avgben/avgsal                          |
| lavgsal  | log(avgsal)                            |
| lenrol   | log(enrol)                             |
| lstaff   | log(staff)                             |
+--------

In [25]:
df

lunch  enrol       staff   exppp   avgsal   avgben  \
distid  schid                                                           
1010.0  4937   48.299999  539.0   71.099998  3586.0  39383.0  13656.0   
2010.0  4959   51.500000   59.0  127.099998  4357.0  25622.0  10004.0   
2020.0  4999   51.200001   82.0  122.599998  5667.0  31318.0  12645.0   
2070.0  597    33.700001  409.0   77.500000  2752.0  27873.0   9657.0   
3010.0  790    11.900000  259.0   87.599998  3253.0  31264.0  10017.0   
...                  ...    ...         ...     ...      ...      ...   
83010.0 2210   62.799999  303.0   64.199997  3049.0  37658.0  12231.0   
        2480   50.000000  281.0   65.500000  2997.0  33885.0  11884.0   
        5101   41.000000  281.0   64.800003  2983.0  36244.0  12120.0   
83060.0 5901    0.000000  506.0   76.599998  3302.0  31165.0  11462.0   
83070.0 2527    0.000000  528.0   58.700001  2482.0  30549.0  10685.0   

                   math4     story4        bs    lavgsal    lenrol    lstaff  \
distid  schid                                                                  
1010.0  4937   68.000000  70.699997  0.346749  10.581090  6.289716  4.264087   
2010.0  4959   40.000000  40.000000  0.390446  10.151207  4.077538  4.844974   
2020.0  4999   57.099998  57.099998  0.403761  10.351949  4.406719  4.808927   
2070.0  597    63.200001  73.500000  0.346464  10.235414  6.013715  4.350278   
3010.0  790    56.400002  64.099998  0.320400  10.350223  5.556828  4.472781   
...                  ...        ...       ...        ...       ...       ...   
83010.0 2210   52.000000  58.000000  0.324792  10.536301  5.713733  4.162003   
        2480   49.000000  74.500000  0.350716  10.430728  5.638355  4.182050   
        5101   70.400002  77.800003  0.334400  10.498029  5.638355  4.171306   
83060.0 5901   53.400002  66.099998  0.367784  10.347051  6.226537  4.338597   
83070.0 2527   43.900002  62.099998  0.349766  10.327087  6.269096  4.072440   

              schid  
distid  schid        
1010.0  4937   4937  
2010.0  4959   4959  
2020.0  4999   4999  
2070.0  597     597  
3010.0  790     790  
...             ...  
83010.0 2210   2210  
        2480   2480  
        5101   5101  
83060.0 5901   5901  
83070.0 2527   2527  

[1848 rows x 13 columns]

In [33]:
#questão (i)
school = pd.DataFrame(df.groupby('distid')['schid'].count())
school['distid'] = school.index
print(f"O valor mínimo de escolas por distrito é {min(school['schid'])} e o máximo é {max(school['schid'])}")
print(f"O valor médio de escolas por distrito é {school['schid'].mean()}.")

O valor mínimo de escolas por distrito é 1 e o máximo é 162
O valor médio de escolas por distrito é 3.441340782122905.


In [34]:
#preparar painel
periodos = pd.Categorical(df['schid'])
df = df.set_index(['distid', 'schid'])
df['schid'] = periodos
df

lunch  enrol       staff   exppp   avgsal   avgben  \
distid  schid                                                           
1010.0  4937   48.299999  539.0   71.099998  3586.0  39383.0  13656.0   
2010.0  4959   51.500000   59.0  127.099998  4357.0  25622.0  10004.0   
2020.0  4999   51.200001   82.0  122.599998  5667.0  31318.0  12645.0   
2070.0  597    33.700001  409.0   77.500000  2752.0  27873.0   9657.0   
3010.0  790    11.900000  259.0   87.599998  3253.0  31264.0  10017.0   
...                  ...    ...         ...     ...      ...      ...   
83010.0 2210   62.799999  303.0   64.199997  3049.0  37658.0  12231.0   
        2480   50.000000  281.0   65.500000  2997.0  33885.0  11884.0   
        5101   41.000000  281.0   64.800003  2983.0  36244.0  12120.0   
83060.0 5901    0.000000  506.0   76.599998  3302.0  31165.0  11462.0   
83070.0 2527    0.000000  528.0   58.700001  2482.0  30549.0  10685.0   

                   math4     story4        bs    lavgsal    lenrol    lstaff  \
distid  schid                                                                  
1010.0  4937   68.000000  70.699997  0.346749  10.581090  6.289716  4.264087   
2010.0  4959   40.000000  40.000000  0.390446  10.151207  4.077538  4.844974   
2020.0  4999   57.099998  57.099998  0.403761  10.351949  4.406719  4.808927   
2070.0  597    63.200001  73.500000  0.346464  10.235414  6.013715  4.350278   
3010.0  790    56.400002  64.099998  0.320400  10.350223  5.556828  4.472781   
...                  ...        ...       ...        ...       ...       ...   
83010.0 2210   52.000000  58.000000  0.324792  10.536301  5.713733  4.162003   
        2480   49.000000  74.500000  0.350716  10.430728  5.638355  4.182050   
        5101   70.400002  77.800003  0.334400  10.498029  5.638355  4.171306   
83060.0 5901   53.400002  66.099998  0.367784  10.347051  6.226537  4.338597   
83070.0 2527   43.900002  62.099998  0.349766  10.327087  6.269096  4.072440   

              schid  
distid  schid        
1010.0  4937   4937  
2010.0  4959   4959  
2020.0  4999   4999  
2070.0  597     597  
3010.0  790     790  
...             ...  
83010.0 2210   2210  
        2480   2480  
        5101   5101  
83060.0 5901   5901  
83070.0 2527   2527  

[1848 rows x 13 columns]

In [36]:
from linearmodels.panel import PooledOLS
#estimando por MQO agrupado
y = df['lavgsal']
x = df[['bs','lenrol','lstaff','lunch']]
X = sm.add_constant(x)
modelo = PooledOLS(y, X)
resultado = modelo.fit()
print(resultado.summary)

                          PooledOLS Estimation Summary                          
Dep. Variable:                lavgsal   R-squared:                        0.4882
Estimator:                  PooledOLS   R-squared (Between):              0.2866
No. Observations:                1848   R-squared (Within):               0.5393
Date:                Sun, May 26 2024   R-squared (Overall):              0.4882
Time:                        02:07:49   Log-likelihood                    689.98
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      439.43
Entities:                         537   P-value                           0.0000
Avg Obs:                       3.4413   Distribution:                  F(4,1843)
Min Obs:                       1.0000                                           
Max Obs:                       162.00   F-statistic (robust):             439.43
                            

In [37]:
y = df['lavgsal']
x = df[['bs', 'lenrol', 'lstaff', 'lunch']]
X = sm.add_constant(x)

#criar e ajustar o modelo Pooled OLS com erros padrão ajustados por clusters
modelo = PooledOLS(y, X)
resultado = modelo.fit(cov_type='clustered', cluster_entity=True)
print(resultado.summary)

                          PooledOLS Estimation Summary                          
Dep. Variable:                lavgsal   R-squared:                        0.4882
Estimator:                  PooledOLS   R-squared (Between):              0.2866
No. Observations:                1848   R-squared (Within):               0.5393
Date:                Sun, May 26 2024   R-squared (Overall):              0.4882
Time:                        02:07:52   Log-likelihood                    689.98
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      439.43
Entities:                         537   P-value                           0.0000
Avg Obs:                       3.4413   Distribution:                  F(4,1843)
Min Obs:                       1.0000                                           
Max Obs:                       162.00   F-statistic (robust):             129.94
                            

In [38]:
# apenas as observações com bs < 0.5
df = df.loc[df['bs'] <= 0.5]
x = df[['bs','lenrol','lstaff','lunch']]
y = df['lavgsal']
X = sm.add_constant(x)
modelo = PooledOLS(y, X)
resultado = modelo.fit(cov_type='clustered', cluster_entity=True)
print(resultado.summary)

                          PooledOLS Estimation Summary                          
Dep. Variable:                lavgsal   R-squared:                        0.4889
Estimator:                  PooledOLS   R-squared (Between):              0.2908
No. Observations:                1844   R-squared (Within):               0.5412
Date:                Sun, May 26 2024   R-squared (Overall):              0.4889
Time:                        02:07:55   Log-likelihood                    707.02
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      439.70
Entities:                         537   P-value                           0.0000
Avg Obs:                       3.4339   Distribution:                  F(4,1839)
Min Obs:                       1.0000                                           
Max Obs:                       161.00   F-statistic (robust):             133.60
                            

In [39]:
#estimando o modelo por Fixed Effects:
x = df[['bs','lenrol','lstaff','lunch']]
y = df['lavgsal']
X = sm.add_constant(x)
#modelo de efeitos fixos
modelo = PanelOLS(y, X, entity_effects=True, drop_absorbed=True).fit()
print(modelo.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                lavgsal   R-squared:                        0.5759
Estimator:                   PanelOLS   R-squared (Between):              0.2774
No. Observations:                1844   R-squared (Within):               0.5759
Date:                Sun, May 26 2024   R-squared (Overall):              0.4555
Time:                        02:07:57   Log-likelihood                    2045.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      442.37
Entities:                         537   P-value                           0.0000
Avg Obs:                       3.4339   Distribution:                  F(4,1303)
Min Obs:                       1.0000                                           
Max Obs:                       161.00   F-statistic (robust):             442.37
                            